In [1]:
import os
from experiment_setup import setups

In [2]:
# download the dataset, the links can be fund in the README
dataset_path = "../datasets/retailrocket"
model_path = "../trained_models"

## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [ ]:
%run ../Preprocess/coveo_ecommerce_preproc.py --path $dataset_path

## Use a specific setup for your dataset

In [3]:
params = setups["retailrocket"]["params_bprmax"]

In [4]:
train_path = os.path.join(dataset_path,"retailrocket_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,"retailrocket_processed_view_test.tsv")

In [5]:
def create_gru4rec_pytorch_script(model_name, train_folder, train_data, test_data, model_path, loss, optim, final_act, layers, batch_size, dropout_p_embed, dropout_p_hidden, learning_rate, n_epochs, m, eval_hidden_reset, use_correct_loss, use_correct_mask_reset):
    checkpoint_dir = f"{model_path}\\{model_name}"
    s_train_full = (
        f"python ..\\GRU4REC-pytorch\\main.py --data_folder {train_folder} "
        f"--train_data {train_data} --valid_data {test_data} --checkpoint_dir {checkpoint_dir} "
        f"--num_layers 1 --embedding_dim {layers} --hidden_size {layers} "
        f"--loss_type {'BPR-max' if loss == 'bpr-max' else 'CrossEntropy'} --final_act {final_act} "
        f"--n_epochs {n_epochs} --batch_size {batch_size} --dropout_input {dropout_p_embed} "
        f"--dropout_hidden {dropout_p_hidden} --lr {learning_rate} --momentum 0.0 "
        f"--optimizer_type {'Adagrad' if optim == 'adagrad' else ''}"
        f"{' --eval_hidden_reset' if eval_hidden_reset else ''}"
        f"{' --use_correct_loss' if use_correct_loss else ''}"
        f"{' --use_correct_mask_reset' if use_correct_mask_reset else ''}"
    )
    s_test_full = s_train_full + f" --is_eval --load_model {checkpoint_dir}\\model_0000{n_epochs-1}.pt --m {m}"
    return s_train_full, s_test_full

In [6]:
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_epochs = 5
m = '1 5 10 20'

## Train & test the out-of-the-box model

In [7]:
train_folder, train_data = '/'.join(train_path.split('/')[:-1]), train_path.split('/')[-1]
test_folder, test_data = '/'.join(test_path.split('/')[:-1]), test_path.split('/')[-1]

In [8]:
train_script_oob, test_script_oob = create_gru4rec_pytorch_script(model_name='gru4rec_pytorch_oob_bprmax', train_folder=train_folder, train_data=train_data, test_data=test_data, model_path=model_path, loss=loss, optim=optim, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=0.0, dropout_p_hidden=0.0, learning_rate=learning_rate, n_epochs=n_epochs, m=m, eval_hidden_reset=False, use_correct_loss=False, use_correct_mask_reset=False)

In [10]:
checkpoint_dir = "../trained_models/gru4rec_pytorch_oob_bprmax"
eval_hidden_reset=False
use_correct_loss=False
use_correct_mask_reset=False

In [12]:
import subprocess

# Comando de entrenamiento
process_train = subprocess.Popen(train_script_oob, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout_train, stderr_train = process_train.communicate()

print("Salida de STDOUT (Entrenamiento):")
print(stdout_train.decode())
print("Salida de STDERR (Entrenamiento):")
print(stderr_train.decode())

# Comando de evaluación
process_test = subprocess.Popen(test_script_oob, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout_test, stderr_test = process_test.communicate()

print("Salida de STDOUT (Evaluación):")
print(stdout_test.decode())
print("Salida de STDERR (Evaluación):")
print(stderr_test.decode())


In [11]:
from model_tracker import track_training_C02_emissions

track_training_C02_emissions(train_script_oob, "gru4rec_pytorch_oob_bprmax", "retailrocket")

[codecarbon WARNING @ 21:19:14] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 21:19:14] [setup] RAM Tracking...
[codecarbon INFO @ 21:19:14] [setup] GPU Tracking...
[codecarbon INFO @ 21:19:14] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 21:19:15] [setup] CPU Tracking...
[codecarbon WARNING @ 21:19:15] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 21:19:17] We saw that you have a 13th Gen Intel(R) Core(TM) i9-13900HX but we don't know it. Please contact us.
[codecarbon INFO @ 21:19:17] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i9-13900HX
[codecarbon WARNING @ 21:19:17] Failed to retrieve gpu information
Traceback (most recent call last):
  File "c:\Users\EVILAB\AppData\Local\Programs\Python\Python312\Lib\site-packages\codecarbon\core\gpu.py", line 238, in get_gpu_details
    devices_info.append(gpu_device.get_gpu_details())
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File 

Salida de STDOUT:                       Args                                             Values
0              hidden_size                                                224
1               num_layers                                                  1
2               batch_size                                                 80
3            dropout_input                                                0.0
4           dropout_hidden                                                0.0
5                 n_epochs                                                  5
6                        m                                               [20]
7           optimizer_type                                            Adagrad
8                final_act                                            elu-0.5
9                       lr                                               0.05
10            weight_decay                                                  0
11                momentum                    

0.0016419794150644483

### Train the out-of-the-box model

In [17]:
os.system(train_script_oob)

sh: 1: python: not found


32512

### Test the out-of-the-box model

In [ ]:
os.system(test_script_oob)

## Train & test inference fix model

In [ ]:
train_script_inffix, test_script_inffix = create_gru4rec_pytorch_script(model_name='gru4rec_pytorch_inffix_bprmax', train_folder=train_folder, train_data=train_data, test_data=test_data, model_path=model_path, loss=loss, optim=optim, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=0.0, dropout_p_hidden=0.0, learning_rate=learning_rate, n_epochs=n_epochs, m=m, eval_hidden_reset=True, use_correct_loss=False, use_correct_mask_reset=False)

### Train the out-of-the-box eval fix model

In [ ]:
os.system(train_script_inffix)

### Test the out-of-the-box eval fix model

In [ ]:
os.system(test_script_inffix)

## Train & test the major fix model

In [ ]:
train_script_majorfix, test_script_majorfix = create_gru4rec_pytorch_script(model_name='gru4rec_pytorch_majorfix_bprmax', train_folder=train_folder, train_data=train_data, test_data=test_data, model_path=model_path, loss=loss, optim=optim, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, n_epochs=n_epochs, m=m, eval_hidden_reset=True, use_correct_loss=True, use_correct_mask_reset=True)

### Train the major fix model

In [ ]:
os.system(train_script_majorfix)

### Test the major fix model

In [ ]:
os.system(test_script_majorfix)